In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Ingesta Gold") \
    .master("spark://localhost:7077")\
    .getOrCreate()

24/10/25 12:26:43 WARN Utils: Your hostname, besgam-Aspire-E1-471 resolves to a loopback address: 127.0.1.1; using 192.168.0.21 instead (on interface wlp9s0)
24/10/25 12:26:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/25 12:26:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.parquet('hdfs://namenode:9000/data/consumidor/strem_reclamacoes/date_ingest=2024-10-25')

In [4]:
df.show()

+--------------------+-------------+-------------+------------+------------------+---+--------------------+--------------------+------+--------------------+
|         nomeEmpresa|       status|tempoResposta|dataOcorrido|            Cidade| UF|              Relato|            Resposta|  Nota|          Comentario|
+--------------------+-------------+-------------+------------+------------------+---+--------------------+--------------------+------+--------------------+
|        Casa & Vídeo|Não Resolvido| no mesmo dia|  24/10/2024|    Rio de Janeiro| RJ|Fiz o pedido de u...|Boa tarde, Carlos...|Nota 3|<não há comentári...|
|     Serasa Experian|Não Resolvido| no mesmo dia|  24/10/2024|    Belo Horizonte| MG|Eles exibem minha...|Oi, LUCAS. Tudo b...|Nota 1|Não resolveram na...|
|Latam Airlines (Tam)|    Resolvido| no mesmo dia|  24/10/2024|        Imperatriz| MA|Prezado(a), como ...|Olá, Informamos q...|Nota 5|Tudo resolvido!! ...|
|Equatorial Maranh...|    Resolvido| no mesmo dia|  24/10/

In [5]:
df.count()

110

In [8]:
df.printSchema()

root
 |-- nomeEmpresa: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tempoResposta: string (nullable = true)
 |-- dataOcorrido: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Relato: string (nullable = true)
 |-- Resposta: string (nullable = true)
 |-- Nota: string (nullable = true)
 |-- Comentario: string (nullable = true)
 |-- date_ingest: date (nullable = true)



In [2]:
df_delta = DeltaTable.forPath(spark, 'hdfs://namenode:9000/data/consumidor/silver')

#df_delta.delete(f"datRefCarga = '{partition}'")

In [12]:
df_delta.delete(f"datRefCarga = '2024-06'")

In [2]:
consumidor = spark.read.format('delta').load("hdfs://namenode:9000/data/consumidor/bronze/")#.where(f"datRefCarga={anoMes}")

In [ ]:
consumidor

In [4]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

In [2]:
spark.sql("show databases").show()

+------------+
|   namespace|
+------------+
|b_consumidor|
|     default|
|g_consumidor|
|s_consumidor|
+------------+



In [2]:
spark.sql("show tables in b_consumidor").show(10, False)

+------------+----------+-----------+
|database    |tableName |isTemporary|
+------------+----------+-----------+
|b_consumidor|consumidor|false      |
+------------+----------+-----------+



In [3]:
spark.sql("select * from b_consumidor.consumidor limit 10").show()

24/10/08 19:37:25 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/10/08 19:37:26 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+------+-----------+------+---+------+----+-----------+-----------+-----------+------------+------------+-----------+----------+---------------+-------------+------------------+-------------+------------+---------------+----+-------+-------------+--------+-------------+---------------+----------+--------+-------------------+--------------+-------------+-----------+
|gestor|canalorigem|regiao| uf|cidade|sexo|faixaetaria|anoabertura|mesabertura|dataabertura|dataresposta|dataanalise|datarecusa|datafinalizacao|prazoresposta|prazoanalisegestor|temporesposta|nomefantasia|segmentomercado|area|assunto|grupoproblema|problema|comocontratou|procurouempresa|respondida|situacao|avaliacaoreclamacao|notaconsumidor|analiserecusa|datrefcarga|
+------+-----------+------+---+------+----+-----------+-----------+-----------+------------+------------+-----------+----------+---------------+-------------+------------------+-------------+------------+---------------+----+-------+-------------+--------+--------

In [63]:
anoMes = "'2024-06'"

In [16]:
consumidor = spark.read.format('delta').load("hdfs://namenode:9000/data/consumidor/silver").where(f"datRefCarga={anoMes}")

In [17]:
consumidor = consumidor.groupBy(col('nomefantasia'), concat(lit('Brasil - '), col('uf')).alias('uf'), col('datRefCarga'))\
                                .agg(count(col('nomefantasia')).alias('qtdReclamcoesUf'))

In [19]:
consumidor.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/gold/reclamacaouf/")

In [18]:
consumidor.show(10)

+--------------------+-----------+-----------+---------------+
|        nomefantasia|         uf|datRefCarga|qtdReclamcoesUf|
+--------------------+-----------+-----------+---------------+
|     Banco Santander|Brasil - MS|    2023-08|             32|
|         Banco Modal|Brasil - PR|    2023-08|              5|
|Banco Santander C...|Brasil - AC|    2023-08|              2|
|         Banco Digio|Brasil - CE|    2023-10|             11|
|     Banco do Brasil|Brasil - MT|    2023-10|             90|
|           Banco PSA|Brasil - PR|    2023-10|              2|
|         Banco Honda|Brasil - TO|    2023-10|              1|
|         BancoSeguro|Brasil - SP|    2024-04|             10|
|       Banco Crefisa|Brasil - CE|    2024-04|              2|
|Banco Mercantil d...|Brasil - PE|    2024-04|              5|
+--------------------+-----------+-----------+---------------+
only showing top 10 rows



In [89]:
df =  consumidor.groupBy(col('nomefantasia'), col('datRefCarga'))\
          .agg(count(col('nomefantasia')).alias('qtdReclamcoes'))\
          .orderBy(col('qtdReclamcoes'), ascending=False)\
          .limit(10)

In [83]:
reclamacaotopten = DeltaTable.forPath(spark, "hdfs://namenode:9000/data/gold/reclamacaotopten/")

In [93]:
anoMes

"'2024-06'"

In [92]:
reclamacaotopten.delete("datRefCarga = '{anoMes}'")

In [91]:
df.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/gold/reclamacaotopten/")

In [11]:
df = consumidor.groupBy(col('nomefantasia'), col('grupoProblema'),col('datRefCarga'))\
          .agg(count(col('nomefantasia')).alias('qtdReclamcoes'))

In [12]:
df.write.partitionBy('datRefCarga').format("delta").mode("append").save("hdfs://namenode:9000/data/gold/grupoproblema/")

In [94]:
consumidor = spark.read.format('delta').load("hdfs://namenode:9000/data/gold/reclamacaotopten/").where(f"datRefCarga={anoMes}")

In [95]:
consumidor.show()

+--------------------+-----------+-------------+
|        nomefantasia|datRefCarga|qtdReclamcoes|
+--------------------+-----------+-------------+
|          Banco Pan |    2024-06|         1208|
|           Banco Bmg|    2024-06|         1080|
|     Banco Santander|    2024-06|          988|
|     Banco do Brasil|    2024-06|          886|
| Banco Itaú Unibanco|    2024-06|          672|
|      Banco Bradesco|    2024-06|          655|
|Banco BV (antigo ...|    2024-06|          462|
|Banco Inter (Banc...|    2024-06|          428|
|      Banco Daycoval|    2024-06|          281|
|Banco C6 Consigna...|    2024-06|          264|
+--------------------+-----------+-------------+



24/08/08 20:38:23 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/08/08 20:38:23 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:716)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:152)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:258)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:168)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:203)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess